In [1]:
from __future__ import division
import torch
import numpy as np
import math
from PIL import Image
from matplotlib import pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


The next few cells contains code for the generative process for observations:

In [40]:
def dirichletDistSample(alphas):
    x = np.zeros([len(alphas),1])
    i = 0
    for a in alphas:
        x[i] = np.random.gamma(a,1)
        i+=1
    sum = np.sum(x)
    return np.divide(x,sum)

In [3]:
def gamma(val):
    return math.factorial(val-1)
    

In [ ]:
for i in range(2):
    x = [100] *6
    plt.plot(dirichletDistSample(x),'ro')
    plt.axis([0,5,0,1])
    plt.show()

In [ ]:
print(np.random.randint(1,16,1))
n_alp = 16
alphas = [.01] * n_alp
sss = np.random.dirichlet(alphas,1)
print(np.asarray(sss))
print(sss.shape)
print(np.random.choice(a = np.arange(0,16), p = np.squeeze(sss)))

In [4]:
#alphas and betas are lists containing the parameter values
n_alp = 16
alphas = [.01] * n_alp
n_bet = 10
betas = [.01] * n_bet

def initializeCanvas( alphas ):
    
    canvas = np.zeros([100,100])
    no_of_topics = len(alphas)
    for r in range(100):
        for c in range(100):
            
            canvas[r,c] = np.random.randint(1,16,1) *16
    
    img = Image.fromarray(canvas)
    img.show()
    
    return canvas
initializeCanvas(alphas)

array([[144.,  80., 224., ..., 176., 176., 144.],
       [ 80., 128., 176., ..., 144., 160., 128.],
       [ 96., 176., 208., ...,  64., 160.,  80.],
       ...,
       [240.,  64., 160., ..., 192.,  64., 160.],
       [ 80., 192.,  48., ..., 128., 128.,  16.],
       [224., 240., 128., ...,  96.,  48., 192.]])

In [5]:
def noOfTopicsInNeighbourhood(r,c,canvas , window):
    
    counter = np.zeros(16)
    val = canvas[r,c]
    #counter = -1
    for i in range(max(0,r-window),min(canvas.shape[0],r+window)):
        for j in range(max(0,c-window),min(canvas.shape[1],c+window)):
            counter[int(canvas[i,j]/16)-1]=counter[int(canvas[i,j]/16)-1]+1
    #print(counter)
    return counter

In [7]:
def generativeDirichlet(alphas,window):
    #window = 1
    init_canvas = initializeCanvas(alphas)
    for n in range(100):
        
        for r in range(100):
            
            for c in range(100):
                
                n_k = noOfTopicsInNeighbourhood(r,c, init_canvas , window)
                update_arr = np.zeros(len(alphas))
                #print(n)
                #update_arr[int(init_canvas[r,c]/16)-1] = n_k
                update_arr = n_k
                cur_alpha = np.squeeze(np.asarray(alphas))+update_arr
                #cur_alpha = np.divide(cur_alpha,sum(cur_alpha))
                #print(cur_alpha)
                dist = np.random.dirichlet(cur_alpha,1)
                init_canvas[r,c] = np.random.choice(a = np.arange(1,17), p = np.squeeze(dist))*16
        #print(n)
    img = Image.fromarray(init_canvas)
    img.show()
    return init_canvas

In [30]:
n_alp = 16
alphas = [.01] * n_alp
cc = generativeDirichlet(alphas,2)

Performing Topic modelling on images (ROST) :

In [4]:
#given a blank dictionary, the number of topics and the vocab, this fuction creates a blank slate to write the topics
#in the randomTopicInit() method
def createTopicDicts(dict_topics , no_of_topics , vocab):
    
    tstr = "topic"
    for i in range(1,no_of_topics+1):
        dict_topics[tstr+str(i)] = {}
        for word in vocab:
            dict_topics[tstr+str(i)][str(word)] = 0
        dict_topics[tstr+str(i)]["total_words"] = 0
    print("created_topic_dict", dict_topics )
    return dict_topics

createDocDicts() : given an empty dictionary creates a dictionay where the keys are the number of documents present in the corpus. This is equivalent to the number of cells in which a single image frame is broken into.

In [5]:

def createDocDicts(dict_docs,doc_counter,no_of_topics):
    dict_docs[str("doc"+str(doc_counter))] = {}
    dict_docs[str("doc"+str(doc_counter))]["total_topics"] = 0
    for i in range(1,no_of_topics+1):
        dict_docs[str("doc"+str(doc_counter))]["topic"+str(i)] = 0
    return dict_docs

createDocDictsROST(): similar to the previous function, but generates a 2d document structure instead of 1D. useful for neighbourhood calculation.

In [6]:
def createDocDictsROST(dict_docs , row , col , no_of_topics):
    dict_docs["doc_{0}_{1}".format(row,col)] = {}
    dict_docs["doc_{0}_{1}".format(row,col)]["total_topics"] = 0
    for i in range(1,no_of_topics+1):
        dict_docs["doc_{0}_{1}".format(row,col)]["topic"+str(i)] = 0
    return dict_docs

randomTopicInit(): given a corpus and number of topics and the vocab, this functions assigns a topic to each of the word at random. This returns a word distribution for each topic (dict_topics), a topic distribution for each word (dict_docs) and a corpus, where each of the words are assigned to a topic (corpus_topic_dict)

In [7]:
#where vocab is a list which contains all the possible words that are there
#in case of the image topic modelling, it will be simple class values.  eg 1,2,.....1024.... etc
#each topic should have all the words present in the vocab
def randomTopicInit(corpus , no_of_topics , vocab):
    #document and topic counter starts from 0
    dict_topics = {}
    dict_docs = {}
    corpus_topic_dict = {}
    doc_counter = 1
    dict_topics = createTopicDicts(dict_topics , no_of_topics , vocab)
    print("TYPE OF CORPUS",type(corpus))
    for doc in corpus:
     
        if str("doc"+str(doc_counter)) not in dict_docs.keys():
            #dict_docs[str("doc"+str(doc_counter))] = {}
            #dict_docs[str("doc"+str(doc_counter))]["total_topics"] = 0
            dict_docs = createDocDicts(dict_docs,doc_counter,no_of_topics)
        doc_topic_list = []
        print(doc)
        for word in doc:
                word = str(word)
                word = word.encode("ascii")
                cur_topic = np.random.randint(1,no_of_topics+1)
                doc_topic_list.append((word,str(cur_topic)))
                if  str("topic"+str(cur_topic)) not in dict_topics.keys():
                    dict_topics[str("topic"+str(cur_topic))] = {}
                    dict_topics[str("topic"+str(cur_topic))]["total_words"] = 0
                #print(dict_topics)
                if word not in dict_topics[str("topic"+str(cur_topic))].keys():
                    dict_topics[str("topic"+str(cur_topic))][word] = 1
                else:
                    dict_topics[str("topic"+str(cur_topic))][word] = dict_topics[str("topic"+str(cur_topic))][word]+1
                dict_topics[str("topic"+str(cur_topic))]["total_words"]+=1
                #print(dict_docs)
                if str("topic"+str(cur_topic)) not in dict_docs[str("doc"+str(doc_counter))].keys():
                    dict_docs[str("doc"+str(doc_counter))][str("topic"+str(cur_topic))] = 1
                else:
                    dict_docs[str("doc"+str(doc_counter))][str("topic"+str(cur_topic))] = dict_docs[str("doc"+str(doc_counter))][str("topic"+str(cur_topic))]+1
                dict_docs[str("doc"+str(doc_counter))]["total_topics"]+=1
    
        if "doc"+str(doc_counter) not in corpus_topic_dict.keys():
            corpus_topic_dict["doc"+str(doc_counter)] = []
        corpus_topic_dict["doc"+str(doc_counter)] = doc_topic_list
        doc_counter+=1
    return dict_topics,dict_docs, corpus_topic_dict

In [8]:
#where vocab is a list which contains all the possible words that are there
#in case of the image topic modelling, it will be simple class values.  eg 1,2,.....1024.... etc
#each topic should have all the words present in the vocab
#neighbourhoods are considered in this version of random initialization
#corpus here is a 2d list.
def randomTopicInitROST(corpus , no_of_topics , vocab):
    #document and topic counter starts from 0
    dict_topics = {}
    dict_docs = {}
    corpus_topic_dict = {}
    doc_counter = 1
    dict_topics = createTopicDicts(dict_topics , no_of_topics , vocab)
    print("TYPE OF CORPUS",type(corpus))
    for row in range(len(corpus)):
        for col in range(len(corpus[0])):
            if "doc_{0}_{1}".format(row,col) not in dict_docs.keys():
                #dict_docs[str("doc"+str(doc_counter))] = {}
                #dict_docs[str("doc"+str(doc_counter))]["total_topics"] = 0
                dict_docs = createDocDictsROST(dict_docs,row , col ,no_of_topics)
            doc_topic_list = []
            #print(doc)
            for word in corpus[row][col]:
                word = str(word)
                word = word.encode("ascii")
                cur_topic = np.random.randint(1,no_of_topics+1)
                doc_topic_list.append((word,str(cur_topic)))
                if  str("topic"+str(cur_topic)) not in dict_topics.keys():
                    dict_topics[str("topic"+str(cur_topic))] = {}
                    dict_topics[str("topic"+str(cur_topic))]["total_words"] = 0
                #print(dict_topics)
                if word not in dict_topics[str("topic"+str(cur_topic))].keys():
                    dict_topics[str("topic"+str(cur_topic))][word] = 1
                else:
                    dict_topics[str("topic"+str(cur_topic))][word] = dict_topics[str("topic"+str(cur_topic))][word]+1
                dict_topics[str("topic"+str(cur_topic))]["total_words"]+=1
                #print(dict_docs)
                if str("topic"+str(cur_topic)) not in dict_docs["doc_{0}_{1}".format(row,col)].keys():
                    dict_docs["doc_{0}_{1}".format(row,col)][str("topic"+str(cur_topic))] = 1
                else:
                    dict_docs["doc_{0}_{1}".format(row,col)][str("topic"+str(cur_topic))] += 1
                dict_docs["doc_{0}_{1}".format(row,col)]["total_topics"]+=1
    
            if "doc_{0}_{1}".format(row,col) not in corpus_topic_dict.keys():
                corpus_topic_dict["doc_{0}_{1}".format(row,col)] = []
            corpus_topic_dict["doc_{0}_{1}".format(row,col)] = doc_topic_list
        doc_counter+=1
    return dict_topics,dict_docs, corpus_topic_dict

In [9]:
#where vocab is a list which contains all the possible words that are there
#in case of the image topic modelling, it will be simple class values.  eg 1,2,.....1024.... etc
#each topic should have all the words present in the vocab
#neighbourhoods are considered in this version of random initialization
#corpus here is a 2d list.
def randomTopicAssignmentROST(corpus , dict_topics, no_of_topics , vocab):
    #document and topic counter starts from 0
    #dict_topics = {}
    dict_docs = {}
    corpus_topic_dict = {}
    doc_counter = 1
    #dict_topics = createTopicDicts(dict_topics , no_of_topics , vocab)
    print("TYPE OF CORPUS",type(corpus))
    for row in range(len(corpus)):
        for col in range(len(corpus[0])):
            if "doc_{0}_{1}".format(row,col) not in dict_docs.keys():
                #dict_docs[str("doc"+str(doc_counter))] = {}
                #dict_docs[str("doc"+str(doc_counter))]["total_topics"] = 0
                dict_docs = createDocDictsROST(dict_docs,row , col ,no_of_topics)
            doc_topic_list = []
            #print(doc)
            for word in corpus[row][col]:
                word = str(word)
                word = word.encode("ascii")
                cur_topic = np.random.randint(1,no_of_topics+1)
                dict_topics['topic'+str(cur_topic)][word]+=1
                dict_topics['topic'+str(cur_topic)]['total_words']+=1
                doc_topic_list.append((word,str(cur_topic)))
                if str("topic"+str(cur_topic)) not in dict_docs["doc_{0}_{1}".format(row,col)].keys():
                    dict_docs["doc_{0}_{1}".format(row,col)][str("topic"+str(cur_topic))] = 1
                else:
                    dict_docs["doc_{0}_{1}".format(row,col)][str("topic"+str(cur_topic))] += 1
                dict_docs["doc_{0}_{1}".format(row,col)]["total_topics"]+=1
    
            if "doc_{0}_{1}".format(row,col) not in corpus_topic_dict.keys():
                corpus_topic_dict["doc_{0}_{1}".format(row,col)] = []
            corpus_topic_dict["doc_{0}_{1}".format(row,col)] = doc_topic_list
        doc_counter+=1
    return dict_topics , dict_docs, corpus_topic_dict

In [100]:
dict_topics, dict_docs ,corpus_topic_dict = randomTopicInitROST(duck_corpus, 3 , vocab)

('created_topic_dict', {'topic1': {'11': 0, '10': 0, '13': 0, '12': 0, '15': 0, '14': 0, '17': 0, '16': 0, '19': 0, '18': 0, 'total_words': 0, '1': 0, '0': 0, '3': 0, '2': 0, '5': 0, '4': 0, '7': 0, '6': 0, '9': 0, '8': 0}, 'topic3': {'11': 0, '10': 0, '13': 0, '12': 0, '15': 0, '14': 0, '17': 0, '16': 0, '19': 0, '18': 0, 'total_words': 0, '1': 0, '0': 0, '3': 0, '2': 0, '5': 0, '4': 0, '7': 0, '6': 0, '9': 0, '8': 0}, 'topic2': {'11': 0, '10': 0, '13': 0, '12': 0, '15': 0, '14': 0, '17': 0, '16': 0, '19': 0, '18': 0, 'total_words': 0, '1': 0, '0': 0, '3': 0, '2': 0, '5': 0, '4': 0, '7': 0, '6': 0, '9': 0, '8': 0}})
('TYPE OF CORPUS', <type 'list'>)


In [159]:
#dict_topics["topic4"]["total_words"]
#dict_docs


updateTopic(): for a given word in a given cell, this function refines the topic for that word based on the word distribution for different topics and topic distribution for the given document.

In [10]:
#takes in a situation and then returns the updated topic of that particular word
#all the other adjustments are done inside
def updateTopic(word ,word_counter, doc ,dict_topics , dict_docs , prev_topic,corpus_topic_dict, alpha , beta , no_of_topics):
    word = word.encode("ascii")
    #print("checking",word)
    topic_probs = np.zeros(no_of_topics+1)
    for i in range(1,no_of_topics+1):
        #print("checking Topic", i)
        #print(dict_topics[str("topic"+str(i))])
        #here n_v_k is calculated from the entire document. And each image is a document.
        #in the paper, a part of the image also called a cell serves as a document
        #so for a given image, there are multiple documents that are spatially divided.
        if word in dict_topics[str("topic"+str(i))].keys():
            n_v_k = (dict_topics[str("topic"+str(i))][word]-1)+beta 
        else:
            n_v_k = beta
        #print(n_v_k)
        n_v_k_sum = (dict_topics[str("topic"+str(i))]["total_words"]-1) + len(dict_topics[str("topic"+str(i))].keys())*beta
        if (i==prev_topic):
            n_k_g = dict_docs[doc][str("topic"+str(i))]-1+alpha
            n_k_g_sum = dict_docs[doc]["total_topics"]-1+ len(dict_docs[doc].keys())*alpha
        else:
            n_k_g = dict_docs[doc][str("topic"+str(i))]+alpha
            n_k_g_sum = dict_docs[doc]["total_topics"]+ len(dict_docs[doc].keys())*alpha

        topic_probs[i] = float(n_v_k/n_v_k_sum)*float(n_k_g/n_k_g_sum)
        
    #print(topic_probs)
    topic_probs = np.divide(topic_probs,sum(topic_probs))
    new_topic = np.random.choice(a=np.arange(1,no_of_topics+1),p = topic_probs[1:]) #here topics start from 0 - no_of_topics so, +1 is addded to the final result to make the 
    #output match the previously followed convention
    
    #print("prev_topic - new_topic" , prev_topic , new_topic)
    dict_docs[doc][str("topic"+str(new_topic))]+=1
    dict_docs[doc][str("topic"+str(prev_topic))]-=1
    if dict_docs[doc][str("topic"+str(prev_topic))]<0:
        exit()
    dict_topics[str("topic"+str(prev_topic))][word]-=1
    dict_topics[str("topic"+str(prev_topic))]["total_words"]-=1
    if word in dict_topics[str("topic"+str(new_topic))].keys():
        dict_topics[str("topic"+str(new_topic))][word]+=1
    else:
        dict_topics[str("topic"+str(new_topic))][word] =1
    dict_topics[str("topic"+str(new_topic))]["total_words"]+=1
    #print(word_counter)
    #print(len(corpus_topic_dict[doc]))
    corpus_topic_dict[doc][word_counter]=(word,str(new_topic))
    return  dict_topics , dict_docs ,corpus_topic_dict
    

countOfWordsWithTopicLabel_i() : given the name of the document, prev_topic, time_frame , spatial_neighbours , dict_topics returns the number of words within the said neighbourhood that has the same topic.

***As of now temporal_neighbours are not considered

In [11]:
def countOfWordsWithTopicLabel_i(doc, i , time_frame , spatial_neighbours , temporal_neighbours , dict_docs):
    index = doc.split('_')[1:3]
    row = int(index[0])-spatial_neighbours
    col = int(index[1])-spatial_neighbours
    total_count = 0
    for r in range(spatial_neighbours*2+1):
        for c in range(spatial_neighbours*2+1):
            
            if 'doc_{0}_{1}'.format(row+r,col+c) in dict_docs.keys():
                #print("Accessing docment "+'doc_{0}_{1}'.format(row+r,col+c) )
                try:
                    int(i)
                    total_count += dict_docs['doc_{0}_{1}'.format(row+r,col+c)]["topic"+str(i)]
                except:
                    total_count += dict_docs['doc_{0}_{1}'.format(row+r,col+c)]["total_topics"]
  
    return total_count
    

In [ ]:
dict_docs

In [155]:
sccc = countOfWordsWithTopicLabel_i('doc_4_3' , 'tota' , 0 , 1, 0 , dict_docs)
print (sccc)

Accessing docment doc_3_2
Accessing docment doc_3_3
Accessing docment doc_3_4
Accessing docment doc_4_2
Accessing docment doc_4_3
Accessing docment doc_4_4
Accessing docment doc_5_2
Accessing docment doc_5_3
Accessing docment doc_5_4
12


updateTopicROST():
Updates the topic of a given word in a given cell/document. Takes into account spatial and temporal neighbourhoods while making the updation.<br/>
word = the word to update.<br/>
word_counter = the counter that keeps track of the position of the word in the document/cell.<br/>
doc = string mentioning the document name e.g. "doc_1_2".<br/>
dict_topics = dictionary with word distribution of topics.<br/>
dict_docs = dictionary with topic distribution in documents/cells.<br/>
spatial_neighbours = neighbourhood info (4 / 8 connected).<br/>
temporal_neighbours = temporal neighbourhood info.<br/>
prev_topic = previous topic for that particular word.<br/>
corpus_topic_dict = dictionary that stores the topic value for all the words in the corpus.<br/>
alpha,beta = dirichlet parameters.<br/>
no_of_topics = number of topics

In [12]:
def updateTopicROST(word , word_counter , doc , dict_topics , dict_docs , spatial_neighbours , temporal_neighbours , prev_topic , corpus_topic_dict , alpha, beta, no_of_topics):
    word = word.encode("ascii")
    time_frame= None
    topic_probs = np.zeros(no_of_topics+1)
    for i in range(1,no_of_topics+1):
        # counts the nuber of words of type 'word' in topic i excluding the given word
        if (i==prev_topic):
            n_v_k = dict_topics['topic'+str(i)][word]-1+beta
            n_v_k_sum = (dict_topics[str("topic"+str(i))]["total_words"]-1) + len(dict_topics[str("topic"+str(i))].keys())*beta

        else:
            n_v_k = dict_topics['topic'+str(i)][word]+beta
            n_v_k_sum = (dict_topics[str("topic"+str(i))]["total_words"]) + len(dict_topics[str("topic"+str(i))].keys())*beta
        
        n_k_g = countOfWordsWithTopicLabel_i(doc , i , time_frame, spatial_neighbours , temporal_neighbours , dict_docs) 
        n_k_g_sum = countOfWordsWithTopicLabel_i( doc , 'total_topics' , time_frame , spatial_neighbours , temporal_neighbours , dict_docs)
        
        if (i==prev_topic):
            n_k_g -= 1
            n_k_g_sum -= 1
        
        numerator = n_k_g+ alpha
        deno = n_v_k_sum + no_of_topics*alpha
        
        topic_probs[i] = (n_v_k/n_v_k_sum)*(n_k_g/n_k_g_sum)
    
    topic_probs = np.divide(topic_probs,sum(topic_probs))
    new_topic = np.random.choice(a=np.arange(1,no_of_topics+1),p = topic_probs[1:]) #here topics start from 0 - no_of_topics so, +1 is addded to the final result to make the 

    #update the topic distribution of document according to the change 
    #in the topic of the word
    dict_docs[doc][str("topic"+str(new_topic))]+=1
    dict_docs[doc][str("topic"+str(prev_topic))]-=1
    #the total count of words in the document remains the same
    
    
    #update the word distribution of topics according to the change in the topic of the word
    dict_topics["topic"+str(prev_topic)][word]-=1
    dict_topics["topic"+str(prev_topic)]["total_words"]-=1
    dict_topics["topic"+str(new_topic)][word]+=1
    dict_topics["topic"+str(new_topic)]["total_words"]+=1
    corpus_topic_dict[doc][word_counter] = (word , str(new_topic))
    return dict_topics, dict_docs,corpus_topic_dict

inferTopicsOfNewObservations(): This function takes in a set of new observations and based on the word distribution per topic and topic distribution per document assigns a topic to a new observation.

In [13]:
def inferTopicsOfNewObservations( corpus , dict_topics , dict_docs ,no_of_topics , alpha , beta):
    key = "doc"
    corpus_topic_dict = {}
    no_of_words = len(dict_topics["topic1"].keys())-1 #total_words is a key which is not a word
    #print(type(no_of_topics))
    #print(no_of_topics)
    prob_arr = np.zeros((no_of_topics+1))
    for i in range(len(corpus)):
        #doc is an nd array
        doc_word_topic_list = []
        for w in corpus[i]:
            #w is the type of word ( it is a number)
            for k in range(1,no_of_topics+1):
                #print('i',i)
                #print('w',w)
                #print('k',k)
                n_v_k = dict_topics["topic"+str(k)][str(w)]+beta
                n_v_k_sum = dict_topics["topic"+str(k)]["total_words"]+no_of_words*beta
                n_k_g = dict_docs["doc"+str(i+1)]["topic"+str(k)] +alpha
                n_k_g_sum = dict_docs["doc"+str(i+1)]["total_topics"]+ no_of_topics*beta
                prob_arr[k] = (n_v_k/n_v_k_sum)*(n_k_g/n_k_g_sum)
                #print('prob_arr',prob_arr)
                prob_arr = np.divide(prob_arr,sum(prob_arr))
                #print('prob_arr_normalized',prob_arr)
            new_topic = np.random.choice(a=np.arange(1,no_of_topics+1),p = prob_arr[1:])
            dict_docs["doc"+str(i+1)]["topic"+str(new_topic)]+=1
            dict_docs["doc"+str(i+1)]["total_topics"]+=1
            dict_topics["topic"+str(new_topic)][str(w)]+=1
            dict_topics["topic"+str(new_topic)]['total_words']+=1
            doc_word_topic_list.append((str(w),str(new_topic)))
        corpus_topic_dict["doc"+str(i+1)] = doc_word_topic_list
    return dict_docs, dict_topics,corpus_topic_dict

updateTopicsFromCorpus(): This function takes in a corpus, the vocab used for the topic modelling, word distribution for topics (dict_topics), topic distribution for documents (dict_docs), the dictionary containing the topic value of each of the words and updates the topic value for each words in the corpus.

updateTopcisFromCorpusROST(): Addded the spatial neighbourhood information

Funtion definition: <br/>
<b>def updateTopicsFromCorpusROST(corpus , vocab ,dict_topics, dict_docs , corpus_topic_dict , spatial_neighbours , temporal_neighbours,no_of_topics , alpha , beta , n_iter):


In [14]:
def updateTopicsFromCorpusROST(corpus , vocab ,dict_topics, dict_docs , corpus_topic_dict , spatial_neighbours , temporal_neighbours,no_of_topics , alpha , beta , n_iter):
    #print("DICT_docsdddddd",dict_docs)
    #print("DICT_topicsddddddd",dict_topics)
    for n in range(n_iter):
        #print("iteration",n)
        #print("corpus_topic_dict",corpus_topic_dict)
        #print(corpus_topic_list[4][12])
        doc_no = 1
        for doc in corpus_topic_dict.keys():
            #print(corpus_topic_dict[doc])
            word_counter = 0
            for word,topic in corpus_topic_dict[doc]:
                #print(word,topic)
                #print(dict_topics["topic"+str(topic)].keys())
                prev_topic = topic
                dict_topics, dict_word , corpus_topic_dict = updateTopicROST(word,word_counter, doc ,dict_topics , dict_docs ,1,0, prev_topic ,corpus_topic_dict, alpha , beta ,no_of_topics)
                word_counter+=1
            doc_no+=1
    return dict_topics, dict_docs , corpus_topic_dict
                           

In [15]:
def updateTopicsFromCorpus(corpus , vocab ,dict_topics, dict_docs , corpus_topic_dict ,no_of_topics , alpha , beta , n_iter):
    #print("DICT_docsdddddd",dict_docs)
    #print("DICT_topicsddddddd",dict_topics)
    for n in range(n_iter):
        #print("iteration",n)
        #print("corpus_topic_dict",corpus_topic_dict)
        #print(corpus_topic_list[4][12])
        doc_no = 1
        for doc in corpus_topic_dict.keys():
            #print(corpus_topic_dict[doc])
            word_counter = 0
            for word,topic in corpus_topic_dict[doc]:
                #print(word,topic)
                #print(dict_topics["topic"+str(topic)].keys())
                prev_topic = topic
                dict_topics, dict_word , corpus_topic_dict = updateTopic(word,word_counter, doc ,dict_topics , dict_docs , prev_topic ,corpus_topic_dict, alpha , beta ,no_of_topics)
                word_counter+=1
            doc_no+=1
    return dict_topics, dict_docs , corpus_topic_dict

generateTopicsFromCorpus(): given corpus, vocab , number of topics, alpha , beta , n_iter,  this functions runs the gibbs sampling for each of the words in the corpus for n_iter number of iterations.

In [16]:
def generateTopicsFromCorpus(corpus , vocab ,no_of_topics , alpha , beta , n_iter ):
    
    dict_topics, dict_docs , corpus_topic_dict = randomTopicInit(corpus, no_of_topics ,vocab)
    #print("DICT_docsdddddd",dict_docs)
    #print("DICT_topicsddddddd",dict_topics)
    for n in range(n_iter):
        #print("iteration",n)
        #print("corpus_topic_dict",corpus_topic_dict)
        #print(corpus_topic_list[4][12])
        doc_no = 1
        for doc in corpus_topic_dict.keys():
            #print(corpus_topic_dict[doc])
            word_counter = 0
            for word,topic in corpus_topic_dict[doc]:
                #print(word,topic)
                #print(dict_topics["topic"+str(topic)].keys())
                prev_topic = topic
                dict_topics, dict_word , corpus_topic_dict = updateTopic(word,word_counter, doc ,dict_topics , dict_docs , prev_topic ,corpus_topic_dict, alpha , beta ,no_of_topics)
                word_counter+=1
            doc_no+=1
    return dict_topics, dict_docs , corpus_topic_dict
    
    
    

generateTopicsFromCorpusROST() : Topic generating function for ROST

In [17]:
def generateTopicsFromCorpusROST(corpus , vocab , no_of_topics , alpha , beta , n_iter ):
    
    spatial_neighbours = 1
    temporal_neighbours = 0
    dict_topics, dict_docs , corpus_topic_dict = randomTopicInitROST(corpus,no_of_topics ,vocab)
    #print("DICT_docsdddddd",dict_docs)
    #print("DICT_topicsddddddd",dict_topics)
    for n in range(n_iter):
        #print("iteration",n)
        #print("corpus_topic_dict",corpus_topic_dict)
        #print(corpus_topic_list[4][12])
        doc_no = 1
        for doc in corpus_topic_dict.keys():
            #print(corpus_topic_dict[doc])
            word_counter = 0
            for word,topic in corpus_topic_dict[doc]:
                #print(word,topic)
                #print(dict_topics["topic"+str(topic)].keys())
                prev_topic = topic
                dict_topics, dict_word , corpus_topic_dict = updateTopicROST(word,word_counter, doc ,dict_topics , dict_docs , spatial_neighbours , temporal_neighbours , prev_topic ,corpus_topic_dict, alpha , beta ,no_of_topics)
                word_counter+=1
            doc_no+=1
    return dict_topics, dict_docs , corpus_topic_dict

In [15]:
dict_topics , dict_docs = generateTopicsFromCorpus(tokenized_data[:3] , 2 , .01 , 1 , 300)

NameError: name 'tokenized_data' is not defined

In [18]:
#given a particular image matrix, this function is responsible for extracting a set (number of features mentioned in 
#thresh) of features (SIFT/SURF) 
import cv2 as cv
from matplotlib import pyplot as plt
def extractSURFFeaturesFromImages( imgfilename , thresh): 
    
    img = cv.imread(imgfilename)
    print(imgfilename)
    surf = cv.xfeatures2d.SURF_create(thresh)
    kp, des = surf.detectAndCompute(img,None)
    #print(len(kp))
    #img2 = cv.drawKeypoints(img,kp,None,(255,0,0),4)
    #plt.imshow(img2),plt.show()
    if (type(des)!=type(None)):
        print(des.shape)
    return (kp, des)
    

createWordClusters() : this function takes in a directory, reads all the images, extract SURF features from them and generates standardized words based on K means clustering. <br>
imgdir : absolute path for the directory that contains the images <br>
no_of_clusters : number of clusters wanted to be generated<br>
thresh : threshold for the SURF feature generation

In [19]:
from sklearn.cluster import KMeans
from random import shuffle
import numpy as np

#given a list of features, this function applies a kmeans clustering algo on them to generate a set of words out of the 
#features obtained from the image
def createClustersFromFeatures(feat_list , no_of_clusters):
    print(len(feat_list))
    #shuffle(feat_list)
    feat_list = feat_list[0:10000]
    print(len(feat_list))
    kmeans = KMeans(n_clusters = no_of_clusters , random_state = 0).fit(feat_list)
    return kmeans

In [20]:
import os
#no_of_clusters translates to the number of words in the vocab for the topic model
def createWordClusters(imgdir , no_of_clusters , thresh):
    
    feat_arr = np.zeros([1,64])
    for root,dirs,files in os.walk(imgdir):
        filelist = files
        break
    #print(filelist)
    count = 0
    for indiv_file in filelist:
        print count
        count+=1
        if count>2000:
            break
        new_path = os.path.join(imgdir,indiv_file)
        (kp,des) = extractSURFFeaturesFromImages(new_path,thresh)
        if (type(des)!=type(None)):
            feat_arr = np.concatenate((feat_arr,des), axis = 0)
        #print(feat_arr.shape)
    cluster_for_img_features = createClustersFromFeatures(feat_arr , no_of_clusters)
    return cluster_for_img_features
        
    
    

In [21]:
#given an imagefile and cluster information and threshold value for SURF 
#this function first extracts SURF features from the image based on the threshold provided
#and then groups them into the closest clusters
def extractWordsFromImages( cluster , imgfile ,thresh):
    (kp,des) = extractSURFFeaturesFromImages(imgfile,thresh)
    words = cluster.predict(des)
    return words,kp
    
    

In [ ]:
words,kp = extractWordsFromImages(clust , 'outdoors.jpg' , 15000)

In [ ]:
corpus = []
corpus.append(words)
dict_topics,dict_docs, corpus_topic_dict = generateTopicsFromCorpus(corpus, vocab , 20 , .001 , 2 , 10)

In [22]:
#generates a random color for each of the topic and returns a
#dictionary where each topic is associated with an RGB tuple
def topicToColor(no_of_topics):
    topic_color_dict = {}
    for i in range(1,no_of_topics+1):
        r = np.random.randint(255)
        g = np.random.randint(255)
        b = np.random.randint(255)
        topic_color_dict[i] = (r,g,b)
    return topic_color_dict

In [23]:
#given an imagefile name, a dictionary that says which word belongs to which topic and a list of 
#keypoints that holds the positions of the above mentioned words, this function plots the words
#onto the image based on the topic for the word (a different color for each topic)
def displayTopicModelling(corpus_topic_dict,no_of_topics,rows , cols ,keypoints,imagefilename ,topic_color_dict):
    #img = cv.imread(imagefilename)
    img = imagefilename
    dpi = 80
    height , width , depth = img.shape
    #topic_color_dict = topicToColor(no_of_topics)
    figsize = width / float(dpi), height /float(dpi)
    lists = corpus_dictToList(corpus_topic_dict,rows, cols)
    #print("Lists",lists)
    for i in range(len(lists)):
        word_list = lists[i]
        cur_keypoints = keypoints[i]
        #print("word_list",len(word_list))
        word_arr = np.asarray(word_list)
        topicwise_kp = []
        for i in range(1,no_of_topics+1):
            #print i
            #print word_arr
            w_arr = np.where(word_arr==i)
            #print w_arr[0]
            #print(keypoints[np.where(word_arr==i)])
            #print ("WARRRRR",len(w_arr[0]))
            #print("keypoints",len(cur_keypoints))
            topicwise_kp = [cur_keypoints[j] for j in w_arr[0]]
            img = cv.drawKeypoints(img,topicwise_kp,None,topic_color_dict[i],4)  
    #cv.namedWindow("Final result")
    ####cv.namedWindow("output", cv.WINDOW_NORMAL) 
    #%matplotlib qt
    
    '''
    fig = plt.figure(figsize = figsize)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(img , cmap='gray')
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.show()
    '''
    
    #display_image_in_actual_size(imagefilename)
    #cv.imshow(img)
    #cv.waitKey(0)
    return img
    

In [24]:
#given an imagefile name, a dictionary that says which word belongs to which topic and a list of 
#keypoints that holds the positions of the above mentioned words, this function plots the words
#onto the image based on the topic for the word (a different color for each topic)
def displayTopicModellingROST(corpus_topic_dict,no_of_topics,rows , cols ,keypoints,imagefilename ,topic_color_dict):
    #img = cv.imread(imagefilename)
    img = imagefilename
    dpi = 80
    height , width , depth = img.shape
    #topic_color_dict = topicToColor(no_of_topics)
    figsize = width / float(dpi), height /float(dpi)
    lists = corpus_dictToListROST(corpus_topic_dict,rows, cols)
    #print("Lists",lists)
    for i in range(len(lists)):
        row_list = lists[i]
        for j in range(len(row_list)):
            cur_keypoints = keypoints[i][j]
            word_list = row_list[j]
            #print("word_list",len(word_list))
            word_arr = np.asarray(word_list)
            topicwise_kp = []
            for k in range(1,no_of_topics+1):
                #print i
                #print word_arr
                w_arr = np.where(word_arr==k)
                #print w_arr[0]
                #print(keypoints[np.where(word_arr==i)])
                #print ("WARRRRR",len(w_arr[0]))
                #print("keypoints",len(cur_keypoints))
                topicwise_kp = [cur_keypoints[l] for l in w_arr[0]]
                img = cv.drawKeypoints(img,topicwise_kp,None,topic_color_dict[k],4)  
    #cv.namedWindow("Final result")
    ####cv.namedWindow("output", cv.WINDOW_NORMAL) 
    #%matplotlib qt
    
    '''
    fig = plt.figure(figsize = figsize)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(img , cmap='gray')
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.show()
    '''
    
    #display_image_in_actual_size(imagefilename)
    #cv.imshow(img)
    #cv.waitKey(0)
    return img
    

In [25]:
list_of_docs = []
#corpus_topic_dict is a dictionary that stores the topic value of each of the words in the corpus
def corpus_dictToList(corpus_topic_dict,rows , cols):
    list_of_docs = []
    for i in range(1,len(corpus_topic_dict.keys())+1):
        doc = "doc"+str(i)
        #print doc
        #print(corpus_topic_dict[doc])
        list_of_keypoints = []
        #print("Type of doc is ", type(corpus_topic_dict[doc]))
        #print("jkasdfhkasdfk",corpus_topic_dict)
        for word in corpus_topic_dict[doc]:
            (wordval,topic) = word
            #print(type(word))
            list_of_keypoints.append(int(topic))
        list_of_docs.append(list_of_keypoints)
        #print(list_of_keypoints)
    return list_of_docs

In [26]:
#the primary difference between this and the non ROST version
#is that this returns a 2D list instead of a 1D list
def corpus_dictToListROST(corpus_topic_dict,rows , cols):
    list_of_docs2D = []
    for r in range(rows):
        row_list = []
        for c in range(cols): 
            doc = "doc_{0}_{1}".format(r,c)
        #print doc
        #print(corpus_topic_dict[doc])
            list_of_keypoints = []
        #print("Type of doc is ", type(corpus_topic_dict[doc]))
        #print("jkasdfhkasdfk",corpus_topic_dict)
            for word in corpus_topic_dict[doc]:
                (wordval,topic) = word
            #print(type(word))
                list_of_keypoints.append(int(topic))
            
            row_list.append(list_of_keypoints)
        list_of_docs2D.append(row_list)
        #print(list_of_keypoints)
    return list_of_docs2D

In [ ]:
sssp = corpus_dictToList(corpus_topic_dict)
sssp , kp_corpus

display_Image_in_actual_size(): This function takes in a image path and reads an image from the path, it also displays the image in its original size.

In [27]:
def display_image_in_actual_size(im_path):

    dpi = 80
    im_data = plt.imread(im_path)
    height, width, depth = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    ax.axis('off')

    # Display the image.
    ax.imshow(im_data, cmap='gray')

    plt.show()

In [ ]:
displayTopicModelling(corpus_topic_dict , 20 ,kp_corpus , 'duck.jpg')

In [ ]:
kp_corpus

In [ ]:
corpus_topic_dict

In [28]:
#vocab = []
#no_of_words = 100
#maxval = max(words)
#for i in range(maxval+1):
#    vocab.append(i)
#print(vocab)
def topicModellingPipeline( imgfilename , word_cluster , vocab, no_of_topics ,thresh):
    
    words,kp = extractWordsFromImages(word_cluster , imgfilename , thresh)
    print("words",words)
    corpus = []
    corpus.append(words)
    dict_topics,dict_docs, corpus_topic_dict = generateTopicsFromCorpus(corpus, vocab , no_of_topics , .01 , 1 , 2)
    final_img = displayTopicModelling(corpus_topic_dict , no_of_topics ,kp , imgfilename)
    return final_img

In [ ]:
vocab = []
no_of_words = 100
no_of_topics = 10
thresh = 6500

#maxval = max(no_of_words)
for i in range(no_of_words):
    vocab.append(i)
print(vocab)
clust = createWordClusters("/home/abhisek/Study/Robotics/yolo/testImgdir" , no_of_words , thresh)
iing = topicModellingPipeline('dota.jpg' , clust , vocab , no_of_topics , thresh)

In [ ]:
lists = corpus_dictToList(corpus_topic_dict)
lists

As of now the topic modelling is considering the entire image for its document which is not the case in the thesis, where a certain boundary of say 1,2,5 pixel width is used to denote a single topic. How to do that?
1. seperate the images into small parts based on the size of the window.
2. run the already implemented in each of these small parts of the image. 
3. Hope for the best.

Remember:
1. corpus = a list of ndarrays, where each ndarray contains the number of words present in a single document. In case the image is broken into multiple small patches where each small patch translates to a document.
2. corpus_topic_dict = a dictionary of documents, where for each document, which word belongs to which topic is given.
3. dict_docs = dictionary of documents(contains the distribution of topics for each document)
4. dict_topics = dictionary of topics (contains the distribution of words for each topic)

In [29]:
#this function will divide a given image into different parts on which the entire previous pipeline will work.
#given an imagename, and a window size, this function will extract small patches of images on which the pipeline 
#be called.
#Edit: The corpus should be 2d, row and columns
#The function now returns a 2d list for the cells. This will be useful for spatial neighbourhoods
import cv2 as cv
from matplotlib import pyplot as plt


def fromImageToCellsToCorpus( cluster ,img , cellsize , thresh):
    img = img
    row,col,_ = img.shape
    prev_r = 0
    doc_dict = {}
    corpus = []
    row_corp = []
    kp_corpus = []
    row_kp = []
    counter = 1
    surf = cv.xfeatures2d.SURF_create(thresh)
    for r in range(cellsize,row,cellsize):
        prev_c = 0
        row_corp = []
        row_kp = []
        for c in range(cellsize,col, cellsize):
            counter+=1
            curblock = img[prev_r:r,prev_c:c,:]
            #do necessary computations
            #doc_dict[base_str+str(counter)] = {}
            #print(prev_r , r)
            #print(prev_c ,c)
            cur_kp,cur_des = surf.detectAndCompute(curblock,None)
            ######img2 = cv.drawKeypoints(curblock,cur_kp,None,(255,0,0),4)
            ######plt.imshow(img2),plt.show()
            cur_words = np.empty( shape=(0, 0) )
            if type(cur_des)!=type(None):
                #dsfsd
                cur_words = cluster.predict(cur_des)

            #prev_r = r
            ##plt.figure()
            ##plt.imshow(curblock)
            #kp_pt[0] is the x ie the column
            #kp_pt[1] is the y ie the row
            for kp_pt in cur_kp:
                temp_c = kp_pt.pt[0]+prev_c
                temp_r = kp_pt.pt[1]+prev_r
                kp_pt.pt = (temp_c,temp_r)
            row_kp.append(cur_kp)
            row_corp.append(cur_words)
            prev_c = c
        corpus.append(row_corp)
        kp_corpus.append(row_kp)
        prev_r = r
    #print("counter",counter)  
    return corpus,kp_corpus

In [30]:
cluster = createWordClusters("/home/abhisek/Study/Robotics/rost-py/val2017" , 20 , 2000)

0
/home/abhisek/Study/Robotics/rost-py/val2017/000000031050.jpg
(434, 64)
1
/home/abhisek/Study/Robotics/rost-py/val2017/000000036844.jpg
(547, 64)
2
/home/abhisek/Study/Robotics/rost-py/val2017/000000042276.jpg
(319, 64)
3
/home/abhisek/Study/Robotics/rost-py/val2017/000000017714.jpg
(190, 64)
4
/home/abhisek/Study/Robotics/rost-py/val2017/000000016228.jpg
(914, 64)
5
/home/abhisek/Study/Robotics/rost-py/val2017/000000064868.jpg
(252, 64)
6
/home/abhisek/Study/Robotics/rost-py/val2017/000000022705.jpg
(210, 64)
7
/home/abhisek/Study/Robotics/rost-py/val2017/000000039405.jpg
(300, 64)
8
/home/abhisek/Study/Robotics/rost-py/val2017/000000055167.jpg
(507, 64)
9
/home/abhisek/Study/Robotics/rost-py/val2017/000000057244.jpg
(1472, 64)
10
/home/abhisek/Study/Robotics/rost-py/val2017/000000047121.jpg
(112, 64)
11
/home/abhisek/Study/Robotics/rost-py/val2017/000000019042.jpg
(758, 64)
12
/home/abhisek/Study/Robotics/rost-py/val2017/000000029984.jpg
(157, 64)
13
/home/abhisek/Study/Robotics/ro

/home/abhisek/Study/Robotics/rost-py/val2017/000000035197.jpg
(621, 64)
112
/home/abhisek/Study/Robotics/rost-py/val2017/000000023034.jpg
(828, 64)
113
/home/abhisek/Study/Robotics/rost-py/val2017/000000017029.jpg
(826, 64)
114
/home/abhisek/Study/Robotics/rost-py/val2017/000000051961.jpg
(1246, 64)
115
/home/abhisek/Study/Robotics/rost-py/val2017/000000018737.jpg
(567, 64)
116
/home/abhisek/Study/Robotics/rost-py/val2017/000000056288.jpg
(238, 64)
117
/home/abhisek/Study/Robotics/rost-py/val2017/000000061960.jpg
(165, 64)
118
/home/abhisek/Study/Robotics/rost-py/val2017/000000063740.jpg
(589, 64)
119
/home/abhisek/Study/Robotics/rost-py/val2017/000000044590.jpg
(378, 64)
120
/home/abhisek/Study/Robotics/rost-py/val2017/000000031093.jpg
(112, 64)
121
/home/abhisek/Study/Robotics/rost-py/val2017/000000036678.jpg
(237, 64)
122
/home/abhisek/Study/Robotics/rost-py/val2017/000000046804.jpg
(512, 64)
123
/home/abhisek/Study/Robotics/rost-py/val2017/000000035963.jpg
(536, 64)
124
/home/abhis

220
/home/abhisek/Study/Robotics/rost-py/val2017/000000029596.jpg
(373, 64)
221
/home/abhisek/Study/Robotics/rost-py/val2017/000000054123.jpg
(782, 64)
222
/home/abhisek/Study/Robotics/rost-py/val2017/000000013177.jpg
(674, 64)
223
/home/abhisek/Study/Robotics/rost-py/val2017/000000036660.jpg
(135, 64)
224
/home/abhisek/Study/Robotics/rost-py/val2017/000000038678.jpg
(225, 64)
225
/home/abhisek/Study/Robotics/rost-py/val2017/000000049259.jpg
(628, 64)
226
/home/abhisek/Study/Robotics/rost-py/val2017/000000036539.jpg
(708, 64)
227
/home/abhisek/Study/Robotics/rost-py/val2017/000000029640.jpg
(874, 64)
228
/home/abhisek/Study/Robotics/rost-py/val2017/000000048555.jpg
(125, 64)
229
/home/abhisek/Study/Robotics/rost-py/val2017/000000052007.jpg
(1296, 64)
230
/home/abhisek/Study/Robotics/rost-py/val2017/000000020107.jpg
(117, 64)
231
/home/abhisek/Study/Robotics/rost-py/val2017/000000059386.jpg
(303, 64)
232
/home/abhisek/Study/Robotics/rost-py/val2017/000000022371.jpg
(112, 64)
233
/home/a

(662, 64)
328
/home/abhisek/Study/Robotics/rost-py/val2017/000000062808.jpg
(674, 64)
329
/home/abhisek/Study/Robotics/rost-py/val2017/000000024021.jpg
(1162, 64)
330
/home/abhisek/Study/Robotics/rost-py/val2017/000000026926.jpg
(387, 64)
331
/home/abhisek/Study/Robotics/rost-py/val2017/000000014007.jpg
(91, 64)
332
/home/abhisek/Study/Robotics/rost-py/val2017/000000060823.jpg
(1754, 64)
333
/home/abhisek/Study/Robotics/rost-py/val2017/000000058029.jpg
(111, 64)
334
/home/abhisek/Study/Robotics/rost-py/val2017/000000023751.jpg
(179, 64)
335
/home/abhisek/Study/Robotics/rost-py/val2017/000000043435.jpg
(86, 64)
336
/home/abhisek/Study/Robotics/rost-py/val2017/000000058384.jpg
(201, 64)
337
/home/abhisek/Study/Robotics/rost-py/val2017/000000041888.jpg
(277, 64)
338
/home/abhisek/Study/Robotics/rost-py/val2017/000000024144.jpg
(538, 64)
339
/home/abhisek/Study/Robotics/rost-py/val2017/000000060932.jpg
(559, 64)
340
/home/abhisek/Study/Robotics/rost-py/val2017/000000017031.jpg
(408, 64)
34

436
/home/abhisek/Study/Robotics/rost-py/val2017/000000055072.jpg
(465, 64)
437
/home/abhisek/Study/Robotics/rost-py/val2017/000000060507.jpg
(518, 64)
438
/home/abhisek/Study/Robotics/rost-py/val2017/000000049091.jpg
(2026, 64)
439
/home/abhisek/Study/Robotics/rost-py/val2017/000000062353.jpg
(1559, 64)
440
/home/abhisek/Study/Robotics/rost-py/val2017/000000033759.jpg
(1291, 64)
441
/home/abhisek/Study/Robotics/rost-py/val2017/000000045550.jpg
(216, 64)
442
/home/abhisek/Study/Robotics/rost-py/val2017/000000023126.jpg
(200, 64)
443
/home/abhisek/Study/Robotics/rost-py/val2017/000000055002.jpg
(170, 64)
444
/home/abhisek/Study/Robotics/rost-py/val2017/000000016249.jpg
(520, 64)
445
/home/abhisek/Study/Robotics/rost-py/val2017/000000042070.jpg
(761, 64)
446
/home/abhisek/Study/Robotics/rost-py/val2017/000000029393.jpg
(724, 64)
447
/home/abhisek/Study/Robotics/rost-py/val2017/000000023359.jpg
(73, 64)
448
/home/abhisek/Study/Robotics/rost-py/val2017/000000052413.jpg
(208, 64)
449
/home/

In [31]:
vocab = []
maxval = 19
for i in range(maxval+1):
    vocab.append(i)
print(vocab)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [92]:
(duck_corpus,kp_corpus) = fromImageToCellsToCorpus(cluster,'duck.jpg',50,2000)

In [93]:
len(kp_corpus)

8

In [ ]:
dict_topics,dict_docs, corpus_topic_dict = generateTopicsFromCorpus(duck_corpus, vocab , 2, .001 , 1 , 10000)

In [ ]:
displayTopicModelling(corpus_topic_dict , 2 ,kp_corpus , 'outdoors.jpg')

In [ ]:
(duck_corpus,kp_corpus) = fromImageToCellsToCorpus(cluster,'outdoors.jpg',140,5000)

Adding the neighbourhood factor of the topic modelling.

frame_list is a list of frames from time = now to time = now-some number ( so that older frames can be updated once in a while) frame_list is a container of fixed length and takes in inputs in FIFO order.

In [ ]:
def topicModelling(frame_list , spatial_neighbourhood , temporal_neighbourhood , dict_topics , dict_docs, ):
    

In [39]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
no_of_topics = 2
topic_color_dict = topicToColor(no_of_topics)
count = 0
cell_size = 50
duck_corpus = []
kp_corpus = []
dict_topics =  {}
dict_docs = {}
corpus_topic_dict = {}
alpha = 0.001
beta = 1
n_iter = 10
init_niter = 100
thresh = 500
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    rows = int(frame.shape[0]/cell_size)
    cols = int(frame.shape[1]/cell_size)
    #print (type(frame))
    if count==0:
    # Our operations on the frame come here
        (duck_corpus,kp_corpus) = fromImageToCellsToCorpus(cluster,frame,cell_size,thresh)
        #print("KPIIIII", len(kp_corpus))
        dict_topics,dict_docs, corpus_topic_dict = generateTopicsFromCorpusROST(duck_corpus, vocab , no_of_topics, alpha , beta , init_niter)
    # Display the resulting frame
    
    else:
        (duck_corpus,kp_corpus) = fromImageToCellsToCorpus(cluster,frame,cell_size,thresh)
        dict_topics , dict_docs,corpus_topic_dict = randomTopicAssignmentROST( duck_corpus,dict_topics,no_of_topics , vocab)
        #print("CORPUS_TOPIC_DICTTTTTTTTTT",corpus_topic_dict)
        dict_topics,dict_docs, corpus_topic_dict = updateTopicsFromCorpusROST(duck_corpus , vocab ,dict_topics, dict_docs , corpus_topic_dict , 1 , 0 , no_of_topics , alpha , beta , n_iter)
    print("PPPPPPP",dict_topics)
    count = count+1
    #print("KPIIIII")
    #print(corpus_topic_dict)
    #print(kp_corpus)
    out_frame = displayTopicModellingROST(corpus_topic_dict , no_of_topics ,rows , cols ,kp_corpus , frame,topic_color_dict)
    cv2.imshow('frame',out_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

('created_topic_dict', {'topic1': {'11': 0, '10': 0, '13': 0, '12': 0, '15': 0, '14': 0, '17': 0, '16': 0, '19': 0, '18': 0, 'total_words': 0, '1': 0, '0': 0, '3': 0, '2': 0, '5': 0, '4': 0, '7': 0, '6': 0, '9': 0, '8': 0}, 'topic2': {'11': 0, '10': 0, '13': 0, '12': 0, '15': 0, '14': 0, '17': 0, '16': 0, '19': 0, '18': 0, 'total_words': 0, '1': 0, '0': 0, '3': 0, '2': 0, '5': 0, '4': 0, '7': 0, '6': 0, '9': 0, '8': 0}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 1, '10': 0, '13': 0, '12': 3, '15': 1, '14': 5, '17': 2, '16': 9, '19': 0, '18': 4, 'total_words': 40, '1': 0, '0': 0, '3': 0, '2': 11, '5': 0, '4': 0, '7': 1, '6': 0, '9': 1, '8': 2}, 'topic2': {'11': 1, '10': 3, '13': 9, '12': 0, '15': 7, '14': 2, '17': 1, '16': 2, '19': 4, '18': 0, 'total_words': 57, '1': 2, '0': 3, '3': 1, '2': 3, '5': 1, '4': 2, '7': 2, '6': 10, '9': 0, '8': 4}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 3, '10': 1, '13': 0, '12': 4, '15': 2, '14': 8, '17': 2, '1

('PPPPPPP', {'topic1': {'11': 16, '10': 12, '13': 0, '12': 47, '15': 12, '14': 57, '17': 13, '16': 173, '19': 22, '18': 58, 'total_words': 728, '1': 14, '0': 1, '3': 13, '2': 214, '5': 5, '4': 18, '7': 14, '6': 3, '9': 21, '8': 15}, 'topic2': {'11': 22, '10': 75, '13': 93, '12': 0, '15': 137, '14': 62, '17': 80, '16': 49, '19': 35, '18': 23, 'total_words': 1078, '1': 40, '0': 48, '3': 19, '2': 36, '5': 18, '4': 42, '7': 80, '6': 123, '9': 25, '8': 71}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 17, '10': 14, '13': 0, '12': 48, '15': 12, '14': 60, '17': 17, '16': 183, '19': 23, '18': 61, 'total_words': 770, '1': 15, '0': 1, '3': 15, '2': 225, '5': 6, '4': 19, '7': 15, '6': 3, '9': 21, '8': 15}, 'topic2': {'11': 24, '10': 79, '13': 98, '12': 0, '15': 145, '14': 68, '17': 84, '16': 52, '19': 38, '18': 24, 'total_words': 1144, '1': 43, '0': 49, '3': 22, '2': 37, '5': 18, '4': 45, '7': 85, '6': 130, '9': 28, '8': 75}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'to

('PPPPPPP', {'topic1': {'11': 30, '10': 21, '13': 1, '12': 101, '15': 17, '14': 86, '17': 44, '16': 303, '19': 42, '18': 94, 'total_words': 1323, '1': 20, '0': 1, '3': 20, '2': 405, '5': 18, '4': 26, '7': 20, '6': 4, '9': 53, '8': 17}, 'topic2': {'11': 39, '10': 136, '13': 195, '12': 3, '15': 256, '14': 100, '17': 166, '16': 72, '19': 72, '18': 37, 'total_words': 2078, '1': 85, '0': 87, '3': 33, '2': 131, '5': 33, '4': 66, '7': 130, '6': 261, '9': 58, '8': 118}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 30, '10': 21, '13': 1, '12': 106, '15': 17, '14': 86, '17': 44, '16': 312, '19': 43, '18': 95, 'total_words': 1359, '1': 20, '0': 1, '3': 20, '2': 420, '5': 19, '4': 27, '7': 20, '6': 4, '9': 56, '8': 17}, 'topic2': {'11': 40, '10': 142, '13': 201, '12': 3, '15': 262, '14': 104, '17': 174, '16': 73, '19': 75, '18': 38, 'total_words': 2141, '1': 88, '0': 90, '3': 33, '2': 133, '5': 34, '4': 68, '7': 132, '6': 268, '9': 61, '8': 122}})
('TYPE OF CORPUS', <type 'list

('PPPPPPP', {'topic1': {'11': 43, '10': 39, '13': 1, '12': 175, '15': 22, '14': 111, '17': 72, '16': 432, '19': 71, '18': 122, 'total_words': 2000, '1': 30, '0': 1, '3': 32, '2': 605, '5': 34, '4': 44, '7': 26, '6': 5, '9': 109, '8': 26}, 'topic2': {'11': 50, '10': 195, '13': 283, '12': 5, '15': 382, '14': 151, '17': 257, '16': 101, '19': 109, '18': 57, 'total_words': 3083, '1': 119, '0': 129, '3': 47, '2': 226, '5': 50, '4': 101, '7': 176, '6': 388, '9': 83, '8': 174}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 45, '10': 43, '13': 1, '12': 178, '15': 22, '14': 113, '17': 74, '16': 443, '19': 73, '18': 128, 'total_words': 2055, '1': 30, '0': 1, '3': 33, '2': 619, '5': 34, '4': 46, '7': 27, '6': 5, '9': 112, '8': 28}, 'topic2': {'11': 50, '10': 197, '13': 286, '12': 5, '15': 388, '14': 152, '17': 262, '16': 103, '19': 109, '18': 58, 'total_words': 3125, '1': 123, '0': 132, '3': 48, '2': 227, '5': 51, '4': 101, '7': 179, '6': 395, '9': 83, '8': 176}})
('TYPE OF CORP

('PPPPPPP', {'topic1': {'11': 60, '10': 63, '13': 2, '12': 230, '15': 34, '14': 141, '17': 105, '16': 594, '19': 86, '18': 178, 'total_words': 2722, '1': 44, '0': 3, '3': 51, '2': 818, '5': 46, '4': 65, '7': 33, '6': 8, '9': 124, '8': 37}, 'topic2': {'11': 62, '10': 251, '13': 345, '12': 8, '15': 491, '14': 200, '17': 332, '16': 132, '19': 131, '18': 94, 'total_words': 4010, '1': 148, '0': 168, '3': 62, '2': 326, '5': 59, '4': 140, '7': 227, '6': 505, '9': 106, '8': 223}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 61, '10': 64, '13': 2, '12': 233, '15': 34, '14': 142, '17': 106, '16': 598, '19': 87, '18': 179, 'total_words': 2749, '1': 45, '0': 3, '3': 51, '2': 827, '5': 46, '4': 66, '7': 33, '6': 8, '9': 126, '8': 38}, 'topic2': {'11': 63, '10': 254, '13': 348, '12': 8, '15': 499, '14': 203, '17': 338, '16': 137, '19': 132, '18': 96, 'total_words': 4082, '1': 149, '0': 170, '3': 63, '2': 339, '5': 62, '4': 143, '7': 231, '6': 512, '9': 107, '8': 228}})
('TYPE OF 

...

('PPPPPPP', {'topic1': {'11': 97, '10': 153, '13': 11, '12': 458, '15': 61, '14': 261, '17': 302, '16': 1560, '19': 161, '18': 313, 'total_words': 5984, '1': 94, '0': 4, '3': 168, '2': 1717, '5': 91, '4': 103, '7': 66, '6': 42, '9': 234, '8': 88}, 'topic2': {'11': 111, '10': 562, '13': 621, '12': 18, '15': 911, '14': 384, '17': 628, '16': 337, '19': 239, '18': 199, 'total_words': 8147, '1': 246, '0': 306, '3': 174, '2': 738, '5': 148, '4': 279, '7': 451, '6': 1024, '9': 188, '8': 583}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 97, '10': 157, '13': 11, '12': 461, '15': 61, '14': 262, '17': 304, '16': 1576, '19': 161, '18': 316, 'total_words': 6044, '1': 96, '0': 4, '3': 169, '2': 1735, '5': 94, '4': 104, '7': 68, '6': 42, '9': 237, '8': 89}, 'topic2': {'11': 112, '10': 570, '13': 624, '12': 19, '15': 917, '14': 388, '17': 633, '16': 339, '19': 241, '18': 199, 'total_words': 8220, '1': 247, '0': 307, '3': 178, '2': 745, '5': 150, '4': 283, '7': 458, '6': 1031, '9':

('PPPPPPP', {'topic1': {'11': 108, '10': 193, '13': 14, '12': 508, '15': 64, '14': 280, '17': 342, '16': 1797, '19': 178, '18': 358, 'total_words': 6764, '1': 98, '0': 4, '3': 189, '2': 1930, '5': 104, '4': 115, '7': 73, '6': 48, '9': 265, '8': 96}, 'topic2': {'11': 140, '10': 698, '13': 703, '12': 26, '15': 1043, '14': 465, '17': 716, '16': 391, '19': 283, '18': 238, 'total_words': 9469, '1': 262, '0': 347, '3': 223, '2': 826, '5': 179, '4': 365, '7': 521, '6': 1147, '9': 231, '8': 665}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 108, '10': 194, '13': 14, '12': 510, '15': 65, '14': 280, '17': 346, '16': 1811, '19': 178, '18': 359, 'total_words': 6799, '1': 98, '0': 4, '3': 191, '2': 1939, '5': 104, '4': 115, '7': 73, '6': 48, '9': 266, '8': 96}, 'topic2': {'11': 145, '10': 707, '13': 708, '12': 28, '15': 1049, '14': 471, '17': 723, '16': 398, '19': 287, '18': 241, 'total_words': 9560, '1': 262, '0': 351, '3': 224, '2': 834, '5': 180, '4': 368, '7': 526, '6': 1154

('PPPPPPP', {'topic1': {'11': 121, '10': 230, '13': 16, '12': 550, '15': 74, '14': 298, '17': 397, '16': 2036, '19': 199, '18': 397, 'total_words': 7523, '1': 105, '0': 4, '3': 209, '2': 2130, '5': 108, '4': 132, '7': 78, '6': 52, '9': 287, '8': 100}, 'topic2': {'11': 174, '10': 844, '13': 778, '12': 41, '15': 1173, '14': 561, '17': 813, '16': 455, '19': 331, '18': 268, 'total_words': 10810, '1': 277, '0': 384, '3': 273, '2': 915, '5': 203, '4': 439, '7': 588, '6': 1272, '9': 264, '8': 757}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 121, '10': 233, '13': 16, '12': 552, '15': 74, '14': 299, '17': 403, '16': 2049, '19': 200, '18': 401, 'total_words': 7570, '1': 106, '0': 4, '3': 211, '2': 2140, '5': 108, '4': 133, '7': 78, '6': 52, '9': 289, '8': 101}, 'topic2': {'11': 175, '10': 848, '13': 786, '12': 41, '15': 1179, '14': 566, '17': 817, '16': 462, '19': 331, '18': 269, 'total_words': 10880, '1': 277, '0': 386, '3': 276, '2': 920, '5': 204, '4': 440, '7': 592, '6'

('PPPPPPP', {'topic1': {'11': 131, '10': 247, '13': 17, '12': 578, '15': 77, '14': 316, '17': 468, '16': 2257, '19': 214, '18': 431, 'total_words': 8256, '1': 126, '0': 6, '3': 232, '2': 2323, '5': 114, '4': 137, '7': 82, '6': 58, '9': 332, '8': 110}, 'topic2': {'11': 191, '10': 916, '13': 835, '12': 48, '15': 1282, '14': 617, '17': 897, '16': 514, '19': 356, '18': 296, 'total_words': 11891, '1': 313, '0': 434, '3': 306, '2': 1010, '5': 222, '4': 466, '7': 661, '6': 1406, '9': 290, '8': 831}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 132, '10': 248, '13': 17, '12': 580, '15': 77, '14': 317, '17': 472, '16': 2271, '19': 216, '18': 433, 'total_words': 8306, '1': 128, '0': 6, '3': 233, '2': 2338, '5': 114, '4': 137, '7': 83, '6': 59, '9': 335, '8': 110}, 'topic2': {'11': 194, '10': 920, '13': 838, '12': 49, '15': 1287, '14': 621, '17': 903, '16': 518, '19': 356, '18': 298, 'total_words': 11960, '1': 315, '0': 438, '3': 308, '2': 1017, '5': 223, '4': 468, '7': 666, '

('PPPPPPP', {'topic1': {'11': 138, '10': 269, '13': 21, '12': 620, '15': 79, '14': 338, '17': 530, '16': 2485, '19': 230, '18': 461, 'total_words': 9009, '1': 148, '0': 6, '3': 259, '2': 2527, '5': 121, '4': 141, '7': 87, '6': 66, '9': 368, '8': 115}, 'topic2': {'11': 210, '10': 980, '13': 893, '12': 54, '15': 1392, '14': 677, '17': 974, '16': 555, '19': 394, '18': 324, 'total_words': 12906, '1': 338, '0': 480, '3': 322, '2': 1106, '5': 245, '4': 495, '7': 732, '6': 1527, '9': 305, '8': 903}})
('TYPE OF CORPUS', <type 'list'>)
('PPPPPPP', {'topic1': {'11': 138, '10': 270, '13': 21, '12': 625, '15': 79, '14': 338, '17': 530, '16': 2496, '19': 231, '18': 462, 'total_words': 9040, '1': 148, '0': 6, '3': 259, '2': 2538, '5': 121, '4': 142, '7': 87, '6': 66, '9': 368, '8': 115}, 'topic2': {'11': 211, '10': 986, '13': 897, '12': 54, '15': 1399, '14': 680, '17': 978, '16': 557, '19': 398, '18': 326, 'total_words': 12963, '1': 339, '0': 482, '3': 322, '2': 1113, '5': 246, '4': 497, '7': 734, '

In [37]:
corpus_topic_dict = inferTopicsOfNewObservations( duck_corpus , dict_topics , dict_docs ,no_of_topics , alpha , beta)

<type 'int'>
2
('i', 6)
('w', 16)
('k', 1)
('prob_arr', array([0.        , 0.02454601, 0.        ]))
('prob_arr_normalized', array([0., 1., 0.]))
('i', 6)
('w', 16)
('k', 2)
('prob_arr', array([0.00000000e+00, 1.00000000e+00, 4.22427035e-05]))
('prob_arr_normalized', array([0.00000000e+00, 9.99957759e-01, 4.22409192e-05]))
('i', 6)
('w', 16)
('k', 1)
('prob_arr', array([0.00000000e+00, 2.45460123e-02, 4.22409192e-05]))
('prob_arr_normalized', array([0.        , 0.99828207, 0.00171793]))
('i', 6)
('w', 16)
('k', 2)
('prob_arr', array([0.00000000e+00, 9.98282069e-01, 4.22427035e-05]))
('prob_arr_normalized', array([0.00000000e+00, 9.99957686e-01, 4.23136079e-05]))
('i', 6)
('w', 2)
('k', 1)
('prob_arr', array([0.00000000e+00, 8.18200409e-03, 4.23136079e-05]))
('prob_arr_normalized', array([0.        , 0.99485506, 0.00514494]))
('i', 6)
('w', 2)
('k', 2)
('prob_arr', array([0.00000000e+00, 9.94855062e-01, 6.22119816e-05]))
('prob_arr_normalized', array([0.00000000e+00, 9.99937470e-01, 6.2

('i', 13)
('w', 0)
('k', 2)
('prob_arr', array([0.        , 0.07574372, 0.00248174]))
('prob_arr_normalized', array([0.        , 0.96827447, 0.03172553]))
('i', 13)
('w', 11)
('k', 1)
('prob_arr', array([0.        , 0.00188815, 0.03172553]))
('prob_arr_normalized', array([0.        , 0.05617221, 0.94382779]))
('i', 13)
('w', 11)
('k', 2)
('prob_arr', array([0.        , 0.05617221, 0.0223357 ]))
('prob_arr_normalized', array([0.       , 0.7154975, 0.2845025]))
('i', 13)
('w', 2)
('k', 1)
('prob_arr', array([0.        , 0.00377631, 0.2845025 ]))
('prob_arr_normalized', array([0.       , 0.0130995, 0.9869005]))
('i', 13)
('w', 2)
('k', 2)
('prob_arr', array([0.        , 0.0130995 , 0.20102127]))
('prob_arr_normalized', array([0.        , 0.06117811, 0.93882189]))
('i', 13)
('w', 2)
('k', 1)
('prob_arr', array([0.        , 0.00377631, 0.93882189]))
('prob_arr_normalized', array([0.        , 0.00400628, 0.99599372]))
('i', 13)
('w', 2)
('k', 2)
('prob_arr', array([0.        , 0.00400628, 0.

('w', 6)
('k', 1)
('prob_arr', array([0.        , 0.10231342, 0.01817259]))
('prob_arr_normalized', array([0.        , 0.84917261, 0.15082739]))
('i', 18)
('w', 6)
('k', 2)
('prob_arr', array([0.        , 0.84917261, 0.0027113 ]))
('prob_arr_normalized', array([0.        , 0.99681728, 0.00318272]))
('i', 18)
('w', 12)
('k', 1)
('prob_arr', array([0.        , 0.07795309, 0.00318272]))
('prob_arr_normalized', array([0.        , 0.96077299, 0.03922701]))
('i', 18)
('w', 12)
('k', 2)
('prob_arr', array([0.00000000e+00, 9.60772986e-01, 6.77825969e-04]))
('prob_arr_normalized', array([0.00000000e+00, 9.99294997e-01, 7.05003273e-04]))
('i', 18)
('w', 10)
('k', 1)
('prob_arr', array([0.        , 0.07308102, 0.000705  ]))
('prob_arr_normalized', array([0.       , 0.9904453, 0.0095547]))
('i', 18)
('w', 10)
('k', 2)
('prob_arr', array([0.00000000e+00, 9.90445300e-01, 6.77825969e-04]))
('prob_arr_normalized', array([0.00000000e+00, 9.99316103e-01, 6.83896835e-04]))
('i', 18)
('w', 10)
('k', 1)
('

ValueError: probabilities are not non-negative

In [42]:
dict_topics['topic1']['16'] = 0
dict_topics['topic1']['total_words']=296

In [45]:
dict_docs['doc1']['topic1'] = 0
dict_docs['doc1']['total_topics'] = 2

In [34]:
dict_topics

{'topic1': {'0': 5,
  '1': 0,
  '10': 6,
  '11': 8,
  '12': 0,
  '13': 1,
  '14': 2,
  '15': 3,
  '16': 63,
  '17': 4,
  '18': 6,
  '19': 5,
  '2': 0,
  '3': 7,
  '4': 1,
  '5': 3,
  '6': 4,
  '7': 2,
  '8': 2,
  '9': 11,
  'total_words': 133},
 'topic2': {'0': 2,
  '1': 4,
  '10': 8,
  '11': 3,
  '12': 5,
  '13': 0,
  '14': 5,
  '15': 41,
  '16': 2,
  '17': 11,
  '18': 1,
  '19': 4,
  '2': 36,
  '3': 3,
  '4': 5,
  '5': 1,
  '6': 26,
  '7': 8,
  '8': 9,
  '9': 4,
  'total_words': 178}}

In [37]:
cap.release()
cv.destroyAllWindows()

In [31]:
cap = cv.VideoCapture(4)

In [57]:
a = [[1,2,3],[4,5,6]]

In [68]:
for i in range(len(a)):
    for j in range(len(a[0])):
        print ("doc_{0}_{1}".format(i,j))

doc_0_0
doc_0_1
doc_0_2
doc_1_0
doc_1_1
doc_1_2
